<a href="https://colab.research.google.com/github/Bannore-Aaditya/Bannore-Aaditya/blob/main/PGMAssignment2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pgmpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pgmpy-0.1.20-py3-none-any.whl (1.9 MB)


In [3]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

Setting models

In [9]:
olympic_model = BayesianModel([('Genetics', 'OlympicTrials'),
                             ('Practice', 'OlympicTrials'),
                             ('OlympicTrials', 'Offer')])


In [16]:
genetics_cpd = TabularCPD(
                variable = 'Genetics',
                variable_card = 2,
                values = [[.2],[.8]])

practice_cpd = TabularCPD(
                variable = 'Practice',
                variable_card = 2,
                values = [[0.7],[0.3]])

In [17]:
offer_cpd = TabularCPD(
                    variable = 'Offer',
                    variable_card = 2,
                    values = [[0.95, 0.8, 0.5],
                             [0.05, 0.2, 0.5]],
                    evidence = ['OlympicTrials'],
                    evidence_card = [3])

olympic_trials_cpd = TabularCPD(
                        variable = 'OlympicTrials', 
                        variable_card = 3,
                        values = [[.5, .8, .8, .9],
                                 [.3, .15, .1, .08],
                                 [.2, .05, .1, .02]],
                        evidence = ['Genetics', 'Practice'],
                        evidence_card = [2,2])

Add the relationships to your models

In [18]:
olympic_model.add_cpds (genetics_cpd, practice_cpd, offer_cpd, olympic_trials_cpd)

Examine the structure of your graph

In [19]:
olympic_model.get_cpds()


[<TabularCPD representing P(Genetics:2) at 0x7fd4c329c690>,
 <TabularCPD representing P(Practice:2) at 0x7fd4c329c150>,
 <TabularCPD representing P(Offer:2 | OlympicTrials:3) at 0x7fd4c14a5dd0>,
 <TabularCPD representing P(OlympicTrials:3 | Genetics:2, Practice:2) at 0x7fd4c14a5090>]

Making inferences

In [21]:
from pgmpy.inference import VariableElimination

olympic_infer = VariableElimination(olympic_model)

In [24]:
prob_offer = olympic_infer.query(variables = ['Offer'])
print(prob_offer)

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8898 |
+----------+--------------+
| Offer(1) |       0.1102 |
+----------+--------------+


In [25]:
prob_offer_good_genes = olympic_infer.query(
                                        variables = ['Offer', 'Genetics'])
print(prob_offer_good_genes)
#print(prob_offer_good_genes['Offer'])

+----------+-------------+-----------------------+
| Offer    | Genetics    |   phi(Offer,Genetics) |
+==========+=============+=======================+
| Offer(0) | Genetics(0) |                0.1684 |
+----------+-------------+-----------------------+
| Offer(0) | Genetics(1) |                0.7214 |
+----------+-------------+-----------------------+
| Offer(1) | Genetics(0) |                0.0316 |
+----------+-------------+-----------------------+
| Offer(1) | Genetics(1) |                0.0786 |
+----------+-------------+-----------------------+


In [26]:
prob_offer_good_genes = olympic_infer.query(
                                        variables = ['Offer', 'OlympicTrials'])
print(prob_offer_good_genes)

+----------+------------------+----------------------------+
| Offer    | OlympicTrials    |   phi(Offer,OlympicTrials) |
+==========+==================+============================+
| Offer(0) | OlympicTrials(0) |                     0.7429 |
+----------+------------------+----------------------------+
| Offer(0) | OlympicTrials(1) |                     0.1010 |
+----------+------------------+----------------------------+
| Offer(0) | OlympicTrials(2) |                     0.0459 |
+----------+------------------+----------------------------+
| Offer(1) | OlympicTrials(0) |                     0.0391 |
+----------+------------------+----------------------------+
| Offer(1) | OlympicTrials(1) |                     0.0252 |
+----------+------------------+----------------------------+
| Offer(1) | OlympicTrials(2) |                     0.0459 |
+----------+------------------+----------------------------+


In [27]:
prob_offer_bad_genes = olympic_infer.query(
                                        variables = ['Offer'], 
                                        evidence = {'Genetics':1})
print(prob_offer_bad_genes)

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.9017 |
+----------+--------------+
| Offer(1) |       0.0983 |
+----------+--------------+


In [28]:
prob_offer_good_genes_did_practice = olympic_infer.query(
                                        variables = ['Offer'], 
                                        evidence = {'Genetics':0, 'Practice':0})
print(prob_offer_good_genes_did_practice)

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8150 |
+----------+--------------+
| Offer(1) |       0.1850 |
+----------+--------------+


In [29]:
prob_good_genes_if_amazing_olympic_trials = olympic_infer.query(
                                        variables = ['Genetics'], 
                                        evidence = {'OlympicTrials':2})
print(prob_good_genes_if_amazing_olympic_trials)

+-------------+-----------------+
| Genetics    |   phi(Genetics) |
+=============+=================+
| Genetics(0) |          0.3377 |
+-------------+-----------------+
| Genetics(1) |          0.6623 |
+-------------+-----------------+


In [30]:
##independencies

olympic_model.get_independencies()

(Practice ⟂ Genetics)
(Practice ⟂ Offer | OlympicTrials)
(Practice ⟂ Offer | OlympicTrials, Genetics)
(Offer ⟂ Practice, Genetics | OlympicTrials)
(Offer ⟂ Genetics | Practice, OlympicTrials)
(Offer ⟂ Practice | OlympicTrials, Genetics)
(Genetics ⟂ Practice)
(Genetics ⟂ Offer | OlympicTrials)
(Genetics ⟂ Offer | Practice, OlympicTrials)